In [ ]:
%load_ext autoreload
%autoreload 2
%cd ..

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import pickle
from typing import Dict, Tuple
from collections import Counter

import matplotlib.pyplot as plt
from matplotlib.dates import date2num
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import scipy.signal
import scipy.fftpack

from util.paths import DATA_PATH
from util.datasets import SlidingWindowDataset, read_physionet_dataset, RespiratoryEventType
from util.filter import apply_butterworth_bandpass_filter, apply_butterworth_lowpass_filter
from util.mathutil import get_peaks, PeakType, cluster_1d, IntRange
from rule_based import detect_respiratory_events
from rule_based.detector import _detect_airflow_resp_events
from util.event_based_metrics import EventBasedConfusionMatrix, get_overlaps

# Makes numpy raise errors instead of outputting warnings
np.seterr('raise')

# Some preparations to pretty-print tensors & ndarrays
np.set_printoptions(edgeitems=10)
np.core.arrayprint._line_width = 400

In [ ]:
dataset_folder = DATA_PATH / "training" / "tr03-0005"
config = SlidingWindowDataset.Config(
    physionet_dataset_folder=dataset_folder,
    downsample_frequency_hz=5,
    time_window_size=pd.Timedelta("5 minutes")
)
sliding_window_dataset = SlidingWindowDataset(config=config, allow_caching=True)

print(f"#Physionet dataset samples: {len(sliding_window_dataset.signals)}")
print(f"#Sliding window positions: {len(sliding_window_dataset)}")
print(f"Timeframe of sliding window positions: {sliding_window_dataset.valid_center_points[-1] - sliding_window_dataset.valid_center_points[0]}")
print(f"Respiratory events list present: {sliding_window_dataset.respiratory_events is not None}")

### Filter events, such that we end up with only _respiratory events_

In [ ]:
annotated_respiratory_events = sliding_window_dataset.respiratory_events

respiratory_event_type_counter = Counter([e.event_type for e in annotated_respiratory_events])
print("Respiratory event types as per annotations:")
print(" - " + "\n - ".join(f"{klass.name}: {cnt}" for klass, cnt in respiratory_event_type_counter.items()))
print()
print(f"{len(annotated_respiratory_events)} annotated respiratory events:")
print(" - " + "\n - ".join([f"#{i}: {evt}" for i, evt in enumerate(annotated_respiratory_events)]))

# Enrich whole sliding window dataset by an events outline
annotated_events_outline_mat = np.zeros(shape=(len(sliding_window_dataset.signals),))
for event in annotated_respiratory_events:
    start_idx = sliding_window_dataset.signals.index.get_loc(event.start, method="nearest")
    end_idx = sliding_window_dataset.signals.index.get_loc(event.end, method="nearest")
    annotated_events_outline_mat[start_idx:end_idx] = 1
annotated_events_outline_series = pd.Series(data=annotated_events_outline_mat, index=sliding_window_dataset.signals.index)
sliding_window_dataset.signals["Annotated respiratory events"] = annotated_events_outline_series

del annotated_events_outline_series, annotated_events_outline_mat

In [ ]:
event_num = 6
event = annotated_respiratory_events[event_num]
print(f"Duration of chosen respiratory event #{event_num}: {(event.end-event.start).total_seconds():.1f}s")
print(f"Type of chosen respiratory event: {event.event_type.name}")

window_center_point = event.start + (event.end-event.start)/2
window_data = sliding_window_dataset.get(center_point=window_center_point)
axes = window_data.signals.plot(figsize=(20, 13), subplots=True)

### Peak detection experiments

In [ ]:
event_num = 0
signal_name = "AIRFLOW"
event = annotated_respiratory_events[event_num]

window_center_point = event.start + (event.end-event.start)/2
window_data = sliding_window_dataset.get(center_point=window_center_point)

#####

kernel_width = int(sliding_window_dataset.config.downsample_frequency_hz*0.7)
peaks = get_peaks(waveform=window_data.signals[signal_name].values, filter_kernel_width=kernel_width)
peaks_mat = np.zeros(shape=(window_data.signals.shape[0],))
for p in peaks:
    peaks_mat[p.start:p.end] = p.extreme_value
peaks_ser = pd.Series(peaks_mat, index = window_data.signals.index, name=f"{signal_name} peaks")

#####

data = pd.concat([window_data.signals[signal_name], peaks_ser, window_data.signals["Annotated respiratory events"]], axis=1).fillna(method="pad")
data.plot(figsize=(20,7), subplots=False)

overall_baseline = np.sqrt(np.mean([np.square(p.extreme_value) for p in peaks]))
plt.axhline(y=overall_baseline, linestyle='--', color="pink")
plt.axhline(y=-overall_baseline, linestyle='--', color="pink")

### AIRFLOW-based detection development

In [ ]:
event_num = 0
signal_name = "AIRFLOW"
event = annotated_respiratory_events[event_num]
# event = detected_but_not_annotated[event_num]

window_center_point = event.start + (event.end-event.start)/2
window_start = window_center_point - sliding_window_dataset.config.time_window_size / 2
window_end = window_center_point + sliding_window_dataset.config.time_window_size / 2

window_data = sliding_window_dataset.get(center_point=window_center_point)

# --------------

kernel_width = int(sliding_window_dataset.config.downsample_frequency_hz*0.7)
peaks = get_peaks(waveform=window_data.signals[signal_name].values, filter_kernel_width=kernel_width)

# ---------------

# overall_baseline = np.sqrt(np.mean([np.square(p.extreme_value) for p in peaks]))
# low_peaks = [p for p in peaks if abs(p.extreme_value)<=overall_baseline]
# low_peaks_mat = np.zeros(shape=(window_data.signals.shape[0],))
# for p in low_peaks:
#     low_peaks_mat[p.start:p.end] = p.extreme_value
# low_peaks_ser = pd.Series(low_peaks_mat, index = window_data.signals.index, name="AIRFLOW below-baseline peaks")

# ---------------

# clusters, coarse_types = _detect_airflow_apnea_areas(window_data.signals["AIRFLOW"].values, sample_frequency_hz=sliding_window_dataset.config.downsample_frequency_hz)
# cluster_mat = np.zeros(shape=(window_data.signals.shape[0],))
# for c in clusters:
#     cluster_mat[c.start:c.end] += 20.0
# clusters_ser = pd.Series(cluster_mat, index=window_data.signals.index, name="AIRFLOW: detected apneas")
# print("Detected apneas:")
# print(" - " + "\n - ".join([f"{t.name}: {c.length/sliding_window_dataset.config.downsample_frequency_hz:.1f}s" for c, t in detected_apnea_events]))

detected_respiratory_events = detect_respiratory_events(signals=window_data.signals, sample_frequency_hz=sliding_window_dataset.config.downsample_frequency_hz, ignore_wake_stages=False)
detected_events_outline_mat = np.zeros(shape=(len(window_data.signals),))
for event in detected_respiratory_events:
    start_idx = window_data.signals.index.get_loc(event.start, method="nearest")
    end_idx = window_data.signals.index.get_loc(event.end, method="nearest")
    detected_events_outline_mat[start_idx:end_idx] = 1
detected_events_outline_series = pd.Series(data=detected_events_outline_mat, index=window_data.signals.index, name="Detected events")

print()
print("Annotated apneas in window:")
annotated_in_window = [e for e in annotated_respiratory_events if e.end > window_start and e.start < window_end]
print(" - " + "\n - ".join([f"{e.event_type.name}: {(e.end-e.start).total_seconds():.1f}s" for e in annotated_in_window]))
print()
print("Detected apneas:")
print(" - " + "\n - ".join([f"{e.event_type.name}: {(e.end-e.start).total_seconds():.1f}s" for e in detected_respiratory_events]))

data = pd.concat([window_data.signals["ABD"], window_data.signals["CHEST"], window_data.signals["AIRFLOW"], window_data.signals["Annotated respiratory events"], detected_events_outline_series], axis=1).fillna(method="pad")
axes = data.plot(figsize=(20,10), subplots=True)
# plt.axhline(y=overall_baseline, linestyle='--', color="pink")
# plt.axhline(y=-overall_baseline, linestyle='--', color="pink")

# print(f"Annotated event position: {window_data.signals.index.get_loc(event.start, method='nearest')}..{window_data.signals.index.get_loc(event.end, method='nearest')}")
None

### Detection run over an entire dataset

In [ ]:
detected_respiratory_events = \
    detect_respiratory_events(sliding_window_dataset.signals,
                              sample_frequency_hz=sliding_window_dataset.config.downsample_frequency_hz,
                              discard_wake_stages=False)
detected_hypopnea_events = [d_ for d_ in detected_respiratory_events if d_.event_type == RespiratoryEventType.Hypopnea]
# detected_apnea_events = [d_ for d_ in detected_apnea_events if d_.apnea_type != ApneaType.Hypopnea]

print()
# print(f"Detected {len(detected_apnea_events)} apnea events & {len(detected_hypopnea_events)} hypopnea events")
print(f"Detected {len(detected_respiratory_events)} respiratory events")
print(f" ..of which are {len(detected_hypopnea_events)} hypopneas")

# Enrich whole sliding window dataset by an events outline
detected_events_outline_mat = np.zeros(shape=(len(sliding_window_dataset.signals),))
for event in detected_respiratory_events:
    start_idx = sliding_window_dataset.signals.index.get_loc(event.start, method="nearest")
    end_idx = sliding_window_dataset.signals.index.get_loc(event.end, method="nearest")
    detected_events_outline_mat[start_idx:end_idx] = 1
detected_events_outline_series = pd.Series(data=detected_events_outline_mat, index=sliding_window_dataset.signals.index)
sliding_window_dataset.signals["Detected respiratory events"] = detected_events_outline_series

del detected_events_outline_series, detected_events_outline_mat

In [ ]:
overlapping_events = []
for a_ in annotated_respiratory_events:
    for d_ in detected_respiratory_events:
        if a_.overlaps(d_):
            if a_ not in overlapping_events:
                overlapping_events += [a_]
            break

detected_but_not_annotated = [d_ for d_ in detected_respiratory_events if not any(a_.overlaps(d_) for a_ in annotated_respiratory_events)]
annotated_but_not_detected = [a_ for a_ in annotated_respiratory_events if not any(d_.overlaps(a_) for d_ in detected_respiratory_events)]

print(f"Number of annotated events: {len(annotated_respiratory_events)}")
print(f"Number of detected events: {len(detected_respiratory_events)}")
print()
print(f"Number of OVERLAPPING events: {len(overlapping_events)}")
print(f"- Coverage of annotated respiratory events {len(overlapping_events)/len(annotated_respiratory_events)*100:.1f}%")
print(f"- Detected events that also appear in annotations: {len(overlapping_events)/len(detected_respiratory_events)*100:.1f}%")
print()

confusion_matrix = EventBasedConfusionMatrix(annotated_events=annotated_respiratory_events, detected_events=detected_respiratory_events)
macro_scores = confusion_matrix.get_macro_scores()
print("Confusion-matrix based macro scores:")
print(f" -> {macro_scores}")

confusion_matrix.plot(title=None)


In [ ]:
event_num = 39
event = annotated_respiratory_events[event_num]
# event = detected_respiratory_events[event_num]
# event = detected_but_not_annotated[event_num]
# event = annotated_but_not_detected[event_num]
# event = detected_hypopnea_events[event_num]

window_center_point = event.start + (event.end-event.start)/2
window_start = window_center_point - sliding_window_dataset.config.time_window_size / 2
window_end = window_center_point + sliding_window_dataset.config.time_window_size / 2

annotated_in_window = [e for e in annotated_respiratory_events if e.end > window_start and e.start < window_end]
detected_in_window = [e for e in detected_respiratory_events if e.end > window_start and e.start < window_end]
print()
print("Annotated respiratory events in window:")
print(" - " + "\n - ".join([f"{e.event_type.name}: {(e.end-e.start).total_seconds():.1f}s" for e in annotated_in_window]))
print()
print("Detected respiratory events in window:")
print(" - " + "\n - ".join([f"{e.event_type.name}: {(e.end-e.start).total_seconds():.1f}s" for e in detected_in_window]))

window_data = sliding_window_dataset.get(center_point=window_center_point)
_ = window_data.signals.plot(figsize=(25, 12), subplots=True)
